In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import re
import time
import sys
import pickle
import datetime

%matplotlib inline

In [2]:
f='2008_rf_data.p'
if os.path.isfile(f):
    df=pickle.load(open(f,'rb'))
else:
    df = pd.read_csv('2008_data.csv')
    df['DepDateTime'] = pd.to_datetime(df['Year']*10000+df['Month']*100+\
                                   df['DayofMonth'],format='%Y%m%d')
    df['DepHour']= pd.cut(df.DepTime,
                      bins=[0,100,200,300,400,500,600,700,800,900,1000,1100,
                            1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,
                            2200,2300,2400],right=False)
    df['DepDelay_30min'] = pd.cut(df.DepDelay, bins=range(-30,2490,30), 
                                  right=False)
    df['ArrDelay_30min'] = pd.cut(df.ArrDelay, bins=range(-30,2490,30), 
                                  right=False)
    carriers=pd.read_csv('carriers.csv')
    airports=pd.read_csv('airports.csv')
    df=pd.merge(df,airports[['iata','airport','lat','long']],left_on='Origin',
                right_on='iata')
    df=pd.merge(df,airports[['iata','airport','lat','long']],left_on='Dest',
                right_on='iata', suffixes=('_Origin','_Dest'))
    df=df[['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'DepDateTime', 'DepHour', 'DepDelay_30min', 'ArrDelay_30min', 
       'airport_Origin', 'lat_Origin', 'long_Origin','airport_Dest', 
       'lat_Dest', 'long_Dest']]
    df['YearMon'] = df.DepDateTime + pd.offsets.MonthBegin(-1)
    pickle.dump(df, open('2008_rf_data.p','wb'))
    df.to_csv('2008_rf_data.csv', index=False)

In [3]:
df.shape

(7009728, 40)

## Create a sample of data

In [8]:
df_sample = df.sample(500)#.query('Month==5')
df_sample.to_csv('2008_01.csv')

In [9]:
df_sample.pivot_table(values='Year', index=['Month','airport_Origin'], 
                      aggfunc='count').reset_index().groupby('Month')\
.count()[['airport_Origin']]

,airport_Origin
Month,
1,35
2,26
3,34
4,30
5,24
6,23
7,31
8,26
9,37
